In [1]:
import os
import sys
import math
import psutil
import sklearn
import humanize
import warnings
import subprocess
import numpy as np
import pandas as pd
import GPUtil as GPU
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.python.client import device_lib


In [2]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [3]:
import keras
import keras.backend as K
from keras.utils import to_categorical
from keras.models import Model, Sequential
from keras.callbacks import ModelCheckpoint
from keras.initializers import glorot_uniform
from keras.layers.core import Permute, Reshape, RepeatVector
from keras.losses import cosine_proximity, categorical_crossentropy
from keras.layers import Input, Dense, Dropout, CuDNNGRU, Embedding, concatenate, Lambda, multiply, merge, Flatten

Using TensorFlow backend.


In [4]:
class SessionDataset:
    """Credit to yhs-968/pyGRU4REC."""    
    def __init__(self, data, sep='\t', user_key='UserId', movie_key='MovieId', time_key='Time', n_samples=-1, movie_map=None, time_sort=False):
        """
        Args:
            data: 讀取好的dataframe
            sep: separator for the csv
            user_key, movie_key, time_key: name of the fields corresponding to the users, movies, time
            n_samples: the number of samples to use. If -1, use the whole dataset.
            movie_map: mapping between movie IDs and movie indices
            time_sort: whether to sort the users by time or not
        """
        self.df = data
        self.user_key = user_key
        self.movie_key = movie_key
        self.time_sort = time_sort #False
        self.add_item_indices(movie_map=movie_map)
        self.df.sort_values([user_key, time_key], inplace=True)

        #Sort the df by time, and then by session ID. That is, df is sorted by session ID and
        #clicks within a session are next to each other, where the clicks within a session are time-ordered.

        self.click_offsets = self.get_click_offsets()
        self.session_idx_arr = self.order_session_idx()
        
    def get_click_offsets(self):
        """
        Return the offsets of the beginning clicks of each session IDs,
        where the offset is calculated against the first click of the first session ID.
        """
        offsets = np.zeros(self.df[self.user_key].nunique() + 1, dtype=np.int32)
        # group & sort the df by user_key and get the offset values
        offsets[1:] = self.df.groupby(self.user_key).size().cumsum()

        return offsets

    def order_session_idx(self):
        """ Order the session indices """
        if self.time_sort:
            # starting time for each users, sorted by session IDs
            users_start_time = self.df.groupby(self.user_key)[self.time_key].min().values
            # order the session indices by session starting times
            session_idx_arr = np.argsort(users_start_time)
        else:
            session_idx_arr = np.arange(self.df[self.user_key].nunique())

        return session_idx_arr
    
    def add_item_indices(self, movie_map=None):
        """ 
        Add item index column named "item_idx" to the df
        Args:
            movie_map (pd.DataFrame): mapping between the item Ids and indices
        """
        if movie_map is None:
            item_ids = self.df[self.movie_key].unique()  # unique item ids
            item2idx = pd.Series(data=np.arange(len(item_ids)),
                                 index=item_ids)
            movie_map = pd.DataFrame({self.movie_key:item_ids,
                                   'item_idx':item2idx[item_ids].values})
        
        self.movie_map = movie_map
        self.df = pd.merge(self.df, self.movie_map, on=self.movie_key, how='inner')
        
    @property    
    def items(self):
        return self.movie_map.MovieId.unique()
        

In [5]:
class SessionDataLoader:
    """Credit to yhs-968/pyGRU4REC."""    
    def __init__(self, dataset, batch_size=50):
        """
        A class for creating session-parallel mini-batches.
        Args:
             dataset (SessionDataset): the session dataset to generate the batches from
             batch_size (int): size of the batch
        """
        self.dataset = dataset
        self.batch_size = batch_size
        self.done_users_counter = 0
        
    def __iter__(self):
        """ Returns the iterator for producing session-parallel training mini-batches.
        Yields:
            input (B,):  Item indices that will be encoded as one-hot vectors later.
            target (B,): a Variable that stores the target item indices
            masks: Numpy array indicating the positions of the users to be terminated
        """

        # initializations
        df = self.dataset.df
        user_key='UserId'
        movie_key='MovieId'
        time_key='TimeStamp'
        self.num_movies = df[movie_key].nunique()+1
        click_offsets = self.dataset.click_offsets
        session_idx_arr = self.dataset.session_idx_arr

        iters = np.arange(self.batch_size)
        maxiter = iters.max()
        start = click_offsets[session_idx_arr[iters]]
        end = click_offsets[session_idx_arr[iters] + 1]
        mask = [] # indicator for the users to be terminated
        finished = False        
        
        

        while not finished:
            minlen = (end - start).min()
            # Item indices (for embedding) for clicks where the first users start
            idx_target = df.item_idx.values[start]
            for i in range(minlen - 1):
                # Build inputs & targets
                idx_input = idx_target
                idx_target = df.item_idx.values[start + i + 1]
                input = idx_input
                target = idx_target
                yield input, target, mask
                
            # click indices where a particular session meets second-to-last element
            start = start + (minlen - 1)
            # see if how many users should terminate
            mask = np.arange(len(iters))[(end - start) <= 1]
            self.done_users_counter = len(mask)
            for idx in mask:
                maxiter += 1
                if maxiter >= len(click_offsets) - 1:
                    finished = True
                    break
                # update the next starting/ending point
                iters[idx] = maxiter
                start[idx] = click_offsets[session_idx_arr[maxiter]]
                end[idx] = click_offsets[session_idx_arr[maxiter] + 1]
        
        print("num_movies ",self.num_movies)
        print("click_offsets ",click_offsets)
        print("session_idx_arr ",session_idx_arr)
        print("iters ",iters)
        print("maxiter " ,maxiter)
        print("start ",start)
        print("end ",end)
        print("mask ",mask)
        print("finished ",finished)

In [6]:
def create_model():   
    emb_size = 50
    hidden_units = 100
    size = emb_size

    inputs = Input(batch_shape=(batch_size, 1, num_movies))
    gru, gru_states = CuDNNGRU(hidden_units, stateful=True, return_state=True)(inputs)# drop1) #
    drop2 = Dropout(0.25)(gru)
    predictions = Dense(num_movies, activation='softmax')(drop2)
    
    model = Model(input=inputs, output=[predictions])
    
    opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss=categorical_crossentropy, optimizer=opt)
    model.summary()

    filepath='DwellTimeModel_checkpoint.h5'
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=2, save_best_only=True, mode='min')
    callbacks_list = []
    return model


def get_states(model):
    return [K.get_value(s) for s,_ in model.state_updates]


def set_states(model, states):
    for (d,_), s in zip(model.state_updates, states):
        K.set_value(d, s)


def get_recall(model, loader, epoch, train_generator_map, recall_k=20):

    test_dataset = SessionDataset(test_data, movie_map=train_generator_map)
    test_generator = SessionDataLoader(test_dataset, batch_size=batch_size)

    n = 0
    suma = 0
    suma_baseline = 0

    for feat, label, mask in test_generator:

        input_oh = to_categorical(feat, num_classes=loader.num_movies) 
        input_oh = np.expand_dims(input_oh, axis=1)
        target_oh = to_categorical(label, num_classes=loader.num_movies)
        pred = model.predict(input_oh, batch_size=batch_size)

        if n%100 == 0:
            try:
                print("{}:{}".format(n, suma/n))
            except:
                pass

        for row_idx in range(feat.shape[0]):
            pred_row = pred[row_idx] 
            label_row = target_oh[row_idx]

            idx1 = pred_row.argsort()[-recall_k:][::-1]
            idx2 = label_row.argsort()[-1:][::-1]

            n += 1
            if idx2[0] in idx1:
                suma += 1

    print("Recall@{} epoch {}: {}".format(recall_k, epoch, suma/n))


def get_mrr(model, loader,epoch,train_generator_map, mrr_k=20):

    test_dataset = SessionDataset(test_data, movie_map = train_generator_map)
    test_generator = SessionDataLoader(test_dataset, batch_size=batch_size)

    n = 0
    suma = 0
    suma_baseline = 0

    for feat, label, mask in test_generator:
        input_oh = to_categorical(feat, num_classes=loader.num_movies) 
        input_oh = np.expand_dims(input_oh, axis=1)
        target_oh = to_categorical(label, num_classes=loader.num_movies)
        pred = model.predict(input_oh, batch_size=batch_size)

        if n%100 == 0:
            try:
                print("{}:{}".format(n, suma/n))
            except:
                pass

        for row_idx in range(feat.shape[0]):
            pred_row = pred[row_idx] 
            label_row = target_oh[row_idx]

            idx1 = pred_row.argsort()[-mrr_k:][::-1]
            idx2 = label_row.argsort()[-1:][::-1]

            n += 1
            if idx2[0] in idx1:
                suma += 1/int((np.where(idx1 == idx2[0])[0]+1))        

    print("MRR@{} epoch {}: {}".format(mrr_k, epoch, suma/n))


def train_model(model, save_weights = False, path_to_weights = True):
    train_dataset = SessionDataset(train_data)
    
    print("train_dataset \n", train_dataset)

    model_to_train = model

    with tqdm(total=train_samples_qty) as pbar: #顯示處理進度條，每一個user處理一次
        for epoch in range(1, 10):  #跑9次
            
            if path_to_weights: # 
                loader = SessionDataLoader(train_dataset, batch_size=batch_size)
                print(epoch, " loader \n", loader)
                
                
            for feat, target, mask in loader:

                input_oh = to_categorical(feat, num_classes=loader.num_movies) 
                input_oh = np.expand_dims(input_oh, axis=1)

                target_oh = to_categorical(target, num_classes=loader.num_movies)

                tr_loss = model_to_train.train_on_batch(input_oh, target_oh)

                real_mask = np.ones((batch_size, 1))
                for elt in mask:
                    real_mask[elt, :] = 0

                hidden_states = get_states(model_to_train)[0]

                hidden_states = np.multiply(real_mask, hidden_states)
                hidden_states = np.array(hidden_states, dtype=np.float32)
                model_to_train.layers[1].reset_states(hidden_states)

                pbar.set_description("Epoch {0}. Loss: {1:.5f}".format(epoch, tr_loss))
                pbar.update(loader.done_users_counter)

            # get metrics for epoch
            get_recall(model_to_train, loader, epoch, train_dataset.movie_map)
            get_mrr   (model_to_train, loader, epoch, train_dataset.movie_map)

            # save model
            if save_weights:
                model_to_train.save('DwellTimeEpoch{}.h5'.format(epoch))


In [7]:
    PATH_TO_TRAIN = 'processed_augmented_train.csv'
    PATH_TO_DEV = 'processed_dev.csv'
    PATH_TO_TEST = 'processed_test.csv'
    train_data = pd.read_csv(PATH_TO_TRAIN, sep='\t', dtype={'MovieId':np.int64})
    dev_data = pd.read_csv(PATH_TO_DEV, sep='\t', dtype={'MovieId':np.int64})
    test_data = pd.read_csv(PATH_TO_TEST, sep='\t', dtype={'MovieId': np.int64})
    
    batch_size = 512
    session_max_len = 100
    embeddingp=False

    num_movies = len(train_data['MovieId'].unique())+1
    print("Unique training movies:", num_movies)

    dev_num_movies = len(dev_data['MovieId'].unique())+1
    print("Unique dev movies:", dev_num_movies)

    test_num_movies = len(test_data['MovieId'].unique())+1
    print("Unique testing movies:", test_num_movies)

    train_samples_qty = len(train_data['UserId'].unique()) 
    print("Training users:", train_samples_qty)

    dev_samples_qty = len(dev_data['UserId'].unique()) 
    print("Dev users:",dev_samples_qty)

    test_samples_qty = len(test_data['UserId'].unique())
    print("Testing users:", test_samples_qty)
    
    train_fraction = 1 # (1 / fraction) most recent session quantity to consider
    dev_fraction = 1

    train_offset_step=train_samples_qty//batch_size
    dev_offset_step=dev_samples_qty//batch_size
    test_offset_step=test_samples_qty//batch_size
    aux = [0]
    aux.extend(list(train_data['MovieId'].unique()))
    itemids = np.array(aux)
    itemidmap = pd.Series(data=np.arange(num_movies), index=itemids) 
    
    model = create_model()
    
    train_model(model)

Unique training movies: 11619
Unique dev movies: 10105
Unique testing movies: 10366
Training users: 19853
Dev users: 5749
Testing users: 5271
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (512, 1, 11619)           0         
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       [(512, 100), (512, 100)]  3516300   
_________________________________________________________________
dropout_1 (Dropout)          (512, 100)                0         
_________________________________________________________________
dense_1 (Dense)              (512, 11619)              1173519   
Total params: 4,689,819
Trainable params: 4,689,819
Non-trainable params: 0
_

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=Tensor("in...)`
  # This is added back by InteractiveShellApp.init_path()
  0%|          | 0/19853 [00:00<?, ?it/s]

train_dataset 
1  loader 
Instructions for updating:
Use tf.cast instead.


Epoch 1. Loss: 4.76964:  98%|█████████▊| 19408/19853 [02:58<00:04, 96.36it/s] 

num_movies  11619
click_offsets  [      0     445     476 ... 1784973 1785011 1785088]
session_idx_arr  [    0     1     2 ... 19850 19851 19852]
iters  [15955 19846 19466 18107 19523 19531 19576 19692 17916 18741 19625 19539
 19746 19661 19458 18760 19735 19481 17958 18374 18486 19728 18706 18859
 19696 17969 19649 19251 19729 19608 19024 19798 19185 19016 19474 19768
 19626 18190 18366 19577 19581 19754 18317 19291 17509 16124 17959 19627
 19492 19628 19706 18994 19502 19223 19685 19193 19799 18995 17249 19711
 17627 17311 19697 18509 19800 19157 19650 19810 18723 19747 17295 19258
 17948 19712 19572 19578 19786 18487 19817 19645 19381 19748 19699 17637
 19792 18227 16712 19546 19825 19637 19427 19662 16109 19811 18555 19687
 19787 18462 18217 18574 19812 16879 18769 19679 19273 19322 19806 19826
 17638 18815 19807 18581 19562 19777 18559 19663 19659 19268 16658 17491
 19613 19749 18854 19111 19739 19592 19759 19820 16984 19835 17441 17520
 19778 19508 17363 19760 17661 19214 18715 1

...

num_movies  11619
click_offsets  [      0     445     476 ... 1784973 1785011 1785088]
session_idx_arr  [    0     1     2 ... 19850 19851 19852]
iters  [15955 19846 19466 18107 19523 19531 19576 19692 17916 18741 19625 19539
 19746 19661 19458 18760 19735 19481 17958 18374 18486 19728 18706 18859
 19696 17969 19649 19251 19729 19608 19024 19798 19185 19016 19474 19768
 19626 18190 18366 19577 19581 19754 18317 19291 17509 16124 17959 19627
 19492 19628 19706 18994 19502 19223 19685 19193 19799 18995 17249 19711
 17627 17311 19697 18509 19800 19157 19650 19810 18723 19747 17295 19258
 17948 19712 19572 19578 19786 18487 19817 19645 19381 19748 19699 17637
 19792 18227 16712 19546 19825 19637 19427 19662 16109 19811 18555 19687
 19787 18462 18217 18574 19812 16879 18769 19679 19273 19322 19806 19826
 17638 18815 19807 18581 19562 19777 18559 19663 19659 19268 16658 17491
 19613 19749 18854 19111 19739 19592 19759 19820 16984 19835 17441 17520
 19778 19508 17363 19760 17661 19214 18715 1

Epoch 9. Loss: 3.24250: : 174735it [49:10, 225.85it/s]

25600:0.1384765625
38400:0.135703125
51200:0.1349609375
64000:0.134140625
76800:0.13364583333333332
89600:0.1338169642857143
102400:0.134775390625
115200:0.13528645833333333
128000:0.135328125
140800:0.13533380681818183
153600:0.135546875
num_movies  7068
click_offsets  [     0     51     72 ... 176713 176718 176744]
session_idx_arr  [   0    1    2 ... 5245 5246 5247]
iters  [5185 4831 4636 4842 4637 4327 4800 5107 4660 5061 4843 4498 3822 5027
 4515 5028 5151 4844 4927 5169 5004 4693 4866 5045 5198 5170 4883 5046
 5075 4884 4814 5186 5171 5047 5216 4673 5217 4832 4257 4499 3954 5172
 4847 4976 4549 5116 5006 5218 5117 5092 4939 5247 5152 4816 5108 5229
 5062 4755 4728 5118 4894 5187 4729 4578 5076 4620 5063 5153 5230 4589
 4533 4928 5231 5199 5030 5048 4674 4231 4886 5219 4912 4817 4551 4694
 4711 4534 5007 5173 4216 5188 4956 5119 4552 5078 4940 5200 4803 4941
 5064 5031 4929 4502 4423 5093 4331 5008 5232 5174 5120 4942 4453 4957
 5201 4503 4470 4895 4913 5202 5079 4677 5220 4383 50

Epoch 9. Loss: 3.24250: : 174735it [53:09, 54.78it/s] 

num_movies  7068
click_offsets  [     0     51     72 ... 176713 176718 176744]
session_idx_arr  [   0    1    2 ... 5245 5246 5247]
iters  [5185 4831 4636 4842 4637 4327 4800 5107 4660 5061 4843 4498 3822 5027
 4515 5028 5151 4844 4927 5169 5004 4693 4866 5045 5198 5170 4883 5046
 5075 4884 4814 5186 5171 5047 5216 4673 5217 4832 4257 4499 3954 5172
 4847 4976 4549 5116 5006 5218 5117 5092 4939 5247 5152 4816 5108 5229
 5062 4755 4728 5118 4894 5187 4729 4578 5076 4620 5063 5153 5230 4589
 4533 4928 5231 5199 5030 5048 4674 4231 4886 5219 4912 4817 4551 4694
 4711 4534 5007 5173 4216 5188 4956 5119 4552 5078 4940 5200 4803 4941
 5064 5031 4929 4502 4423 5093 4331 5008 5232 5174 5120 4942 4453 4957
 5201 4503 4470 4895 4913 5202 5079 4677 5220 4383 5080 5049 4915 4369
 5189 5233 5203 4836 5032 5065 3987 4236 4758 4697 3998 5204 5205 4472
 5121 5033 5122 4993 5154 5123 4943 4820 5066 5206 4130 4930 5207 4747
 5208 5034 4145 4019 4698 4959 5175 4917 5221 5155 4897 4504 4563 4792
 5176 51